# Impute gene activity weights using magic

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Import packages

In [5]:
import anndata as ad
import scanpy as sc
import os
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import scvelo as scv

In [6]:
import graphtools
import magic

In [7]:
scv.set_figure_params()
sc.settings.set_figure_params(dpi=100, facecolor='white', frameon=False)

In [8]:
from utils import *

In [9]:
my_cmap = get_continuous_cmap(['D6D6D6', '5D4FF4'])

## Setup paths

In [10]:
data_path = '/lustre/groups/ml01/workspace/laura.martens/moretti_colab'
save_path = os.path.join(data_path, 'biological_analysis')
scvelo_path = os.path.join(data_path, 'scvelo')

In [11]:
fig_path = '/lustre/groups/ml01/workspace/laura.martens/moretti_colab/panels'
fig_path = os.path.join(fig_path, 'epicardial')
sc.settings.figdir = fig_path

# Load data

In [12]:
adata = sc.read('/lustre/groups/ml01/workspace/laura.martens/moretti_colab/scglue/metacells.h5ad')

In [13]:
gene_act = get_act(adata)

In [14]:
act_mat = pd.DataFrame(gene_act.X.A, index=gene_act.obs_names.values, columns=gene_act.var_names.values)

In [15]:
act_mat = act_mat.fillna(0)

# Run magic on gene activities

In [54]:
magic_op = magic.MAGIC(knn=4, n_jobs=8)

In [55]:
magic_op

MAGIC(knn=4, knn_max=12, n_jobs=8)

In [20]:
g = graphtools.Graph(data=gene_act.obsm['X_glue'], knn=15, knn_max=15, decay=1, thresh=1e-4)

In [21]:
g.data = act_mat.values

In [58]:
imputed = magic_op.fit_transform(act_mat, genes=act_mat.columns.to_list(), graph=g)

Calculating MAGIC...
  Running MAGIC on 35499 cells and 20010 genes.
  Using precomputed graph and diffusion operator...
  Running MAGIC with `solver='exact'` on 20010-dimensional data may take a long time. Consider denoising specific genes with `genes=<list-like>` or using `solver='approximate'`.
  Calculating imputation...
  Calculated imputation in 68.75 seconds.
Calculated MAGIC in 119.05 seconds.


In [ ]:
adata.obsm['activities_imputed'] = imputed.values

In [65]:
adata.write('/lustre/groups/ml01/workspace/laura.martens/moretti_colab/scglue/metacells_imputed.h5ad')